In [ ]:
import random

In [ ]:
img_tech = ["Gamma Correction", "Gaussian Blur", "Unsharp Masking", "Histogram Equalisation", "Contrast Stretching"]

In [ ]:
def parameter_tech(tech):
  if tech == "Gamma Correction":
    gamma = random.uniform(0.5, 2.0)
    return {"gamma":gamma}
  elif tech == "Gaussian Blur":
    return {"sigma": random.uniform(0.5, 5.0)}
  elif tech == "Unsharp Masking":
    return {"amount": random.uniform(0.5, 2.0)}
  elif tech == "Histogram Equalisation":
    return {}
  elif tech == "Contrast Stretching":
    return
  else:
    return {}

In [ ]:
def ran_pipeline(min_len=4, max_len=6):
  len = random.randint(min_len, max_len)
  pipeline = []
  for x in range(len):
    tech = random.choice(img_tech)
    para = parameter_tech(tech)
    pipeline.append({"tech":tech, "params":parameter_tech(tech)})
  return pipeline

In [ ]:
random_pipeline = ran_pipeline()
print(random_pipeline)

[{'tech': 'Gaussian Blur', 'params': {'sigma': 3.109908727356519}}, {'tech': 'Gamma Correction', 'params': {'gamma': 1.1440545008444796}}, {'tech': 'Gaussian Blur', 'params': {'sigma': 1.906983770334503}}, {'tech': 'Gamma Correction', 'params': {'gamma': 0.5074905670003411}}]
